In [ ]:
import pandas as pd
import torch
import os
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Load the model meta-llama2
model_id_1= "meta-llama/Llama-2-7b-chat-hf"

# Load the model meta-llama3.2
model_id_2= "meta-llama/Llama-3.2-3B-Instruct"

tokenizer_1= AutoTokenizer.from_pretrained(model_id_1)
model_1 = AutoModelForCausalLM.from_pretrained(model_id_1, device_map='auto', torch_dtype=torch.float16)

tokenizer_2= AutoTokenizer.from_pretrained(model_id_2)
model_2 = AutoModelForCausalLM.from_pretrained(model_id_2, device_map='auto', torch_dtype=torch.float16)

# Load the CSV file containing sample bios
bio_df = pd.read_csv("accountant_samples.csv") #path to Bios sample 

# Read the job ad prompt from a text file
with open("prompts/prompt_template_jobAD.txt", "r") as f:  #path to job ad prompt text file 
    job_ad_prompt = f.read()

# Load the LLaMA_2 model with text generation pipeline
pipe1 = pipeline(
    "text-generation",
    model=model_1,
    tokenizer=tokenizer_1
   # device=0 if torch.cuda.is_available() else -1  
)

# Load the LLaMA_3.2 model with text generation pipeline
pipe2 = pipeline(
    "text-generation",
    model=model_2,
    tokenizer=tokenizer_2
   # device=0 if torch.cuda.is_available() else -1  
)
# --- Generate Outputs ---
def format_prompt(template, bio, profession, gender):
    return f"{template.format(bio=bio, gender=gender, profession=profession)}" 



Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.54it/s]
Device set to use cpu
Device set to use cpu


In [ ]:
# Generate text for each bio (max_new_tokens=512))
generated_texts = []
for i, row in bio_df.iterrows():  
    bio = row["hard_text"]      # bio text column
    profession = row["profession"]
    gender = row["gender"] 
    prompt = format_prompt(job_ad_prompt, bio, profession, gender)
    
    response = pipe1(prompt, max_new_tokens=512, temperature=0.0, do_sample=False)[0]["generated_text"]
    generated_texts.append({
        "profession": profession,
        "bio": bio,
        "generated_text": response
    })
    print(f"\n--- Result for Bio #{i+1} ---\n{response}\n")


# Create the subfolder if it doesn't exist
os.makedirs("job_ads/jobADs_accountant", exist_ok=True)

# Save each job ad as a separate file inside the folder
for i, ad in enumerate(generated_texts, start=1):
    filename = f"job_ads/jobADs_accountant/job_ad_{i}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        
            f.write(f"\n{ad['generated_text']}\n")
            f.write("\n" + "-"*80 + "\n\n")

#### Generation of job ADs using model 1, max token 2048

In [ ]:
# Generate text for each bio (max_new_tokens=2048)
generated_texts = []
for i, row in bio_df.iterrows():  
    bio = row["hard_text"]      # bio text column
    profession = row["profession"]
    gender = row["gender"] 
    prompt = format_prompt(job_ad_prompt, bio, profession, gender)
    
    response = pipe1(prompt, max_new_tokens=2048, temperature=0.0, do_sample=False)[0]["generated_text"]
    generated_texts.append({
        "profession": profession,
        "bio": bio,
        "generated_text": response
    })
    print(f"\n--- Result for Bio #{i+1} ---\n{response}\n")


# Create the subfolder if it doesn't exist
os.makedirs("job_ads/jobADs_len2048", exist_ok=True)

# Save each job ad as a separate file inside the folder
for i, ad in enumerate(generated_texts, start=1):
    filename = f"job_ads/jobADs_len2048/job_ad_{i}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        
            f.write(f"\n{ad['generated_text']}\n")
            f.write("\n" + "-"*80 + "\n\n")

#### Generation of job ADs using model 2, max token 2048

In [3]:
# Generate text for each bio (max_new_tokens=2048)
generated_texts = []
for i, row in bio_df.iterrows():  
    bio = row["hard_text"]      # bio text column
    profession = row["profession"]
    gender = row["gender"] 
    prompt = format_prompt(job_ad_prompt, bio, profession, gender)
    
    response = pipe2(prompt, max_new_tokens=2048, temperature=0.0, do_sample=False)[0]["generated_text"]
    generated_texts.append({
        "profession": profession,
        "bio": bio,
        "generated_text": response
    })
    print(f"\n--- Result for Bio #{i+1} ---\n{response}\n")


# Create the subfolder if it doesn't exist
os.makedirs("job_ads/jobADs_Llama3.2", exist_ok=True)

# Save each job ad as a separate file inside the folder
for i, ad in enumerate(generated_texts, start=1):
    filename = f"job_ads/jobADs_Llama3.2/job_ad_{i}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        
            f.write(f"\n{ad['generated_text']}\n")
            f.write("\n" + "-"*80 + "\n\n")

/home/shilpi/Documents/sem3/P2_BiasMitigation_DataAugmentation/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/shilpi/Documents/sem3/P2_BiasMitigation_DataAugmentation/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- Result for Bio #1 ---

You are a professional recruiter. Write a compelling and inclusive job advertisement tailored to the following candidate's background:
Profession: accountant
Gender: Female
Bio: "She was raised up in a remote rural area where most of the basic needs, education materials, and spiritual guidance were offered by charity organizations. This grew her deep appreciation for non-profits and she was inspired from her early childhood years to take part in empowering people from humble backgrounds. Lucy joined the SAI/Servant Forge team in 2015 as an accounts intern and acquired skills that she now applies to serve as accounts and administration assistant. She is thankful that SAI/ Servant Forge has given her an opportunity to fulfill her dream of serving the community."

Make sure the job ad highlights the candidate's strengths and is suitable for a relevant job position in a well-known company.


"Generated job ad:"
**Job Title: Financial Analyst**

**Company:** Serv

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- Result for Bio #2 ---

You are a professional recruiter. Write a compelling and inclusive job advertisement tailored to the following candidate's background:
Profession: accountant
Gender: Female
Bio: "She has served as General Manager in companies in the industrial and real estate sector and has experience in companies that represent foreign homes in Costa Rica. She also has extensive experience in tenders contracted through the Law of Administrative Contracting."

Make sure the job ad highlights the candidate's strengths and is suitable for a relevant job position in a well-known company.


"Generated job ad:"
**Job Title:** Financial Manager - International Business Operations

**Company:** XYZ Corporation

**Location:** San José, Costa Rica

**Job Type:** Full-time

**About Us:**

At XYZ Corporation, we are a leading global company with a strong presence in the international market. We are seeking an experienced and skilled Financial Manager to join our team in San José, Costa

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...


--- Result for Bio #7 ---

You are a professional recruiter. Write a compelling and inclusive job advertisement tailored to the following candidate's background:
Profession: accountant
Gender: Female
Bio: "Since 2015 she has been the Director of the Michael Hughes Foundation, an Australia Not for Profit company with a focus on raising public awareness of sudden cardiac arrest, promotion of first aid training and placement of defibrillators in the community. Between 2007-2013 Michelle was an accountant at Goodwin Chivas & Co Chartered Accountants before becoming company Manager from 2013-2017. Michelle has a Bachelor of Business, Accounting and Finance from the University of Technology, Sydney (2003-2005) and a Graduate Diploma of Chartered Accounting from the Institute of Chartered Accountants in Australia, 2007-2009."

Make sure the job ad highlights the candidate's strengths and is suitable for a relevant job position in a well-known company.


"Generated job ad:"
**Job Title: Finan

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- Result for Bio #8 ---

You are a professional recruiter. Write a compelling and inclusive job advertisement tailored to the following candidate's background:
Profession: accountant
Gender: Male
Bio: "He is Accredited in Business Valuation (ABV), Certified in Financial Forensics (CFF), and a Chartered Global Management Accountant (CGMA) with more than 40 years of finance, management and accounting experience in a variety of industries – as an expert witness, a CFO in private industry, and a manager with Price Waterhouse. Barnes has taught courses for The National Judicial College, AICPA, Price Waterhouse, Hastings College of Law, and the College of Advocacy. He has been a speaker at AICPA’s National Business Valuation Conferences and the CalCPA Education Foundation’s Advanced Business Litigation Institutes. Barnes has received awards from both AICPA and the Education Foundation for his work in business valuation education.""

Make sure the job ad highlights the candidate's strength

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- Result for Bio #9 ---

You are a professional recruiter. Write a compelling and inclusive job advertisement tailored to the following candidate's background:
Profession: accountant
Gender: Female
Bio: "She graduated with a Bachelor of Science and a Bachelor of Arts in Accounting from Richard Stockton College. She enjoys spending time with her family, baking and basketball."

Make sure the job ad highlights the candidate's strengths and is suitable for a relevant job position in a well-known company.


"Generated job ad:"
**Job Title: Financial Analyst - Join Our Team!**

**Company:** XYZ Corporation, a leading provider of financial solutions and services

**Location:** New York, NY

**Job Type:** Full-time

**About Us:**

At XYZ Corporation, we're passionate about delivering exceptional financial solutions that drive business growth and success. We're committed to fostering a diverse and inclusive work environment where everyone feels valued and empowered to thrive.

**Job Summary